In [15]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['figure.figsize'] = (15, 10)

def consolidate_summary_evaluations(BASE_PATH, name, RATIOS):
    all_dfs = []
    for ratio in RATIOS:
        csv_path = os.path.join(BASE_PATH, f"{ratio}_{name}", f"{ratio}_summary_evaluation.csv")
        df = pd.read_csv(csv_path)
        df.insert(0, 'Ratio', ratio)
        all_dfs.append(df)
        print(f"{ratio} 데이터 로드 완료 ({len(df)} 행)")

    combined_df = pd.concat(all_dfs, ignore_index=True)
    combined_df = combined_df[combined_df['Type'] == 'Mean'].drop(columns=['Type'])
    os.makedirs(f'{BASE_PATH}/combined_result_{name}/combined_result', exist_ok=True)
    output_path = f"{BASE_PATH}/combined_result_{name}/combined_result/consolidated_summary_evaluation.csv"
    combined_df.to_csv(output_path, index=False)
    return combined_df

def create_combined_plots(BASE_PATH, name, RATIOS):
    plot_types = ['Learning Curve', 'AUC', 'Confusion Matrix', 'Feature Importance', 'Calibration Curve', 'Class Prediction Error']
    for plot_type in plot_types:
        create_combined_plot_for_type(plot_type, name, BASE_PATH, RATIOS)

def create_combined_plot_for_type(plot_type, name, BASE_PATH, RATIOS):
    filename_mapping = {
        'Learning Curve': 'learning.png',
        'AUC': 'auc.png', 
        'Confusion Matrix': 'confusion_matrix.png',
        'Feature Importance': 'feature.png',
        'Calibration Curve': 'calibration.png',
        'Class Prediction Error': 'error.png'
    }
    
    filename_suffix = filename_mapping.get(plot_type)
    if not filename_suffix:
        return
    
    MODELS = []
    folder_path = os.path.join(BASE_PATH, f"{RATIOS[0]}_{name}")
    excel_files = [f for f in os.listdir(folder_path) if f.endswith(('csv'))]
    for file in excel_files:
        model_name = file.split('_')[1]
        if model_name != 'summary':
            MODELS.append(model_name)
 
    fig, axes = plt.subplots(len(RATIOS), len(MODELS), figsize=(len(MODELS)*4, len(RATIOS)*4))
    fig.suptitle(f'{plot_type} - All Models and Ratios', fontsize=18, y=0.98)

    for i, ratio in enumerate(RATIOS):
        for j, model in enumerate(MODELS):
            ax = axes[i, j]
            image_path = os.path.join(BASE_PATH, f"{ratio}_{name}", "plots", f"{ratio}_{model}_{filename_suffix}")
            
            if os.path.exists(image_path):
                img = plt.imread(image_path)
                ax.imshow(img)
                ax.axis('off')
            else:
                ax.text(0.5, 0.5, f'No {plot_type}\nfor {model}\n({ratio})', ha='center', va='center', 
                        transform=ax.transAxes, fontsize=8, bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgray"))
                ax.axis('off')
            
            if i == 0:
                ax.set_title(f'{model}', fontsize=10, pad=10, weight='bold')
            if j == 0:
                ax.set_ylabel(f'{ratio}', fontsize=12, rotation=0, labelpad=30, weight='bold')
    
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    
    os.makedirs(f'{BASE_PATH}/combined_result_{name}/combined_result', exist_ok=True)
    output_path = f"{BASE_PATH}/combined_result_{name}/combined_result/combined_{plot_type.replace(' ', '_').lower()}.png"
    plt.savefig(output_path, dpi=200, bbox_inches='tight')
    plt.close()

def create_performance_heatmap(df, BASE_PATH, name, RATIOS):
    if df is None:
        return
    
    metrics = ['Accuracy', 'AUC', 'Recall', 'Prec.', 'F1', 'Kappa', 'MCC']
    
    for metric in metrics:
        if metric in df.columns:
            pivot_df = df.pivot_table(index='Ratio', columns='Model', values=metric, aggfunc='mean')
            
            pivot_df = pivot_df.reindex(RATIOS)
            
            plt.figure(figsize=(14, len(RATIOS)*3))
            sns.heatmap(pivot_df, annot=True, cmap='RdYlBu_r', fmt='.3f', cbar_kws={'label': metric}, linewidths=0.5)
            plt.title(f'{metric} Performance Heatmap - All Models and Ratios', fontsize=14, pad=20)
            plt.xlabel('Model', fontsize=12)
            plt.ylabel('Data Ratio', fontsize=12)
            plt.xticks(rotation=45, ha='right')
            plt.tight_layout()
            
            os.makedirs(f'{BASE_PATH}/combined_result_{name}/heatmap', exist_ok=True)
            output_path = f"{BASE_PATH}/combined_result_{name}/heatmap/heatmap_{metric.lower().replace('.', '')}.png"
            plt.savefig(output_path, dpi=200, bbox_inches='tight')
            plt.close()
            print(f"{metric} 히트맵 저장: {output_path}")

def create_blended_plots(BASE_PATH, name, RATIOS):
    plot_types = ['learning', 'auc', 'confusion_matrix', 'calibration', 'error']
    
    for plot_type in plot_types:
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        fig.suptitle(f'Blended Model - {plot_type.replace("_", " ").title()}', fontsize=16, y=0.9)
        axes = axes.flatten()
        
        for i, ratio in enumerate(RATIOS):
            if i < len(axes):
                ax = axes[i]
                image_path = os.path.join(BASE_PATH, f"{ratio}_{name}", "plots_blended", f"{ratio}_{name}_blended_{plot_type}.png")

                try:
                    if os.path.exists(image_path):
                        img = plt.imread(image_path)
                        ax.imshow(img)
                        ax.set_title(f'{ratio}', fontsize=12, weight='bold')
                    else:
                        ax.text(0.5, 0.5, f'No blended\n{plot_type}\nfor {ratio}', ha='center', va='center', transform=ax.transAxes)
                except Exception:
                    ax.text(0.5, 0.5, f'Error loading\nblended {plot_type}\nfor {ratio}', ha='center', va='center', transform=ax.transAxes)
                
                ax.axis('off')
        
        plt.tight_layout(rect=[0, 0, 1, 0.96])
        os.makedirs(f'{BASE_PATH}/combined_result_{name}/blended_model', exist_ok=True)
        output_path = f"{BASE_PATH}/combined_result_{name}/blended_model/blended_{plot_type}.png"
        plt.savefig(output_path, dpi=200, bbox_inches='tight')
        plt.close()

In [ ]:
BASE_PATH1 = '../result/FTO_FP'
RATIOS = ['1x', '3x', '5x', '10x', '20x', '50x']
combined_df = consolidate_summary_evaluations(BASE_PATH1, 'fp', RATIOS)
create_combined_plots(BASE_PATH1, 'fp', RATIOS)
#create_blended_plots(BASE_PATH1, 'fp', RATIOS)
create_performance_heatmap(combined_df, BASE_PATH1, 'fp', RATIOS)

for BASE_PATH in ['../result/FTO_FP+MD', '../result/FTO_FP+weka_MD']:
    RATIOS = ['5x', '10x', '20x']
    for name in ['mdF','mdT']:
        combined_df = consolidate_summary_evaluations(BASE_PATH, name, RATIOS)
        create_combined_plots(BASE_PATH, name, RATIOS)
        #create_blended_plots(BASE_PATH, name, RATIOS)
        create_performance_heatmap(combined_df, BASE_PATH, name, RATIOS)

In [16]:
BASE_PATH1 = '../result/FTO_Final'

RATIOS = ['5x', '10x']
combined_df = consolidate_summary_evaluations(BASE_PATH1, 'w3D', RATIOS)
create_combined_plots(BASE_PATH1, 'w3D', RATIOS)
create_performance_heatmap(combined_df, BASE_PATH1, 'w3D', RATIOS)

5x 데이터 로드 완료 (14 행)
10x 데이터 로드 완료 (14 행)
Accuracy 히트맵 저장: ../result/FTO_Final/combined_result_w3D/heatmap/heatmap_accuracy.png
AUC 히트맵 저장: ../result/FTO_Final/combined_result_w3D/heatmap/heatmap_auc.png
Recall 히트맵 저장: ../result/FTO_Final/combined_result_w3D/heatmap/heatmap_recall.png
Prec. 히트맵 저장: ../result/FTO_Final/combined_result_w3D/heatmap/heatmap_prec.png
F1 히트맵 저장: ../result/FTO_Final/combined_result_w3D/heatmap/heatmap_f1.png
Kappa 히트맵 저장: ../result/FTO_Final/combined_result_w3D/heatmap/heatmap_kappa.png
MCC 히트맵 저장: ../result/FTO_Final/combined_result_w3D/heatmap/heatmap_mcc.png
